In [36]:
import os
import numpy as np
import pandas as pd
from dfply import *
from ChainLadder import ChainLadder
import matplotlib.pyplot as plt
from datetime import datetime

In [37]:
pd.options.display.float_format = '{:,.2f}'.format

In [129]:
자동차_원수_계약 = pd.read_excel('./data/자동차_충격시나리오/자동차_원수_계약.xlsx', dtype={'UY': str, 'FY': str, 'AY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str})
자동차_원수_보상 = pd.read_excel('./data/자동차_충격시나리오/자동차_원수_보상.xlsx', dtype={'UY': str, 'FY': str, 'AY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str})
자동차_출재_계약 = pd.read_excel('./data/자동차_충격시나리오/자동차_출재_계약.xlsx', dtype={'UY': str, 'FY': str, 'AY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str})
자동차_출재_보상 = pd.read_excel('./data/자동차_충격시나리오/자동차_출재_보상.xlsx', dtype={'UY': str, 'FY': str, 'AY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str})
보험가격_위험계수 = pd.read_excel('./data/자동차_충격시나리오/자동차_충격시나리오_기초정보.xlsx', sheet_name='보험가격위험계수')
준비금_위험계수 = pd.read_excel('./data/자동차_충격시나리오/자동차_충격시나리오_기초정보.xlsx', sheet_name='준비금위험계수')
기준손해율 = pd.read_excel('./data/자동차_충격시나리오/자동차_충격시나리오_기초정보.xlsx', sheet_name='기준손해율')
자동차_상품군정보 = pd.read_excel('./data/자동차_충격시나리오/자동차_상품군정보.xlsx', sheet_name='상품군분류', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str})
자동차_분류 = pd.read_excel('./data/자동차_충격시나리오/자동차_상품군정보.xlsx', sheet_name='KICS분류')

In [132]:
자동차_원수_계약_가공 = 자동차_원수_계약 >> left_join(자동차_상품군정보, on=['BSC_CVR_CD', 'PDGR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_분류, on=X.PDGR_NM)

자동차_원수_보상_가공 = 자동차_원수_보상 >> mask(X.AY != " ", ~X.AY.isna(), ~X.UY.isna()) \
    >> mutate(DY=X.FY.astype(int)-X.AY.astype(int)+1) \
    >> mask(X.DY >= 1) \
    >> mutate(LOSS=X.PYN_BNF+X.OST_BNF) \
    >> left_join(자동차_상품군정보, on=['BSC_CVR_CD', 'PDGR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_분류, on=X.PDGR_NM)

자동차_출재_계약_가공 = 자동차_출재_계약 >> left_join(자동차_상품군정보, on=['BSC_CVR_CD', 'PDGR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_분류, on=X.PDGR_NM)

자동차_출재_보상_가공 = 자동차_출재_보상 >> mask(X.AY != " ", ~X.AY.isna(), ~X.UY.isna()) \
    >> mutate(DY=X.FY.astype(int)-X.AY.astype(int)+1) \
    >> mask(X.DY >= 1) \
    >> mutate(RN_LOSS=X.RN_PYN_BNF+X.RN_OST_BNF) \
    >> left_join(자동차_상품군정보, on=['BSC_CVR_CD', 'PDGR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_분류, on=X.PDGR_NM)

In [135]:
# 추가진전율
기준년도 = '2018'

result = []
for 보종코드 in 준비금_위험계수['보종코드']:

    n = 5
    시작년도 = str(int(기준년도) - n + 1)

    # 삼각형 생성
    자동차_원수_보상_보종 = 자동차_원수_보상_가공 >> mask(X.AT_PDGR_CD == 보종코드, X.AY >= 시작년도, X.DY <= n)

    삼각형_손해액_보종 = 자동차_원수_보상_보종.pivot_table(index='AY', columns='DY', values='LOSS', aggfunc=np.sum)
    삼각형_손해액_보종.index.name = 'Accident'; 삼각형_손해액_보종.columns.name = 'Development'

    삼각형_보험금_보종 = 자동차_원수_보상_보종.pivot_table(index='AY', columns='DY', values='PYN_BNF', aggfunc=np.sum)
    삼각형_보험금_보종.index.name = 'Accident'; 삼각형_보험금_보종.columns.name = 'Development'

    자동차_원수_계약_보종 = 자동차_원수_계약_가공 \
        .query('FY >= @시작년도') \
        .query('AT_PDGR_CD == @보종코드') \
        .groupby(['AT_PDGR_CD', 'FY'])[['ELP_PRM']] \
        .sum() \
        .reset_index()

    cl_loss = ChainLadder(삼각형_손해액_보종, type='incr')
    cl_paid = ChainLadder(삼각형_보험금_보종, type='incr')

    위험계수 = 준비금_위험계수.loc[준비금_위험계수['보종코드'] == 보종코드, '준비금위험계수'].values[0]
    진전계수 = cl_loss.dev_factor('weighted average', 1)
    최종손해액_손해액진전_보종 = cl_loss.develop(진전계수)['Ult'].to_frame('Value')
    최종손해액_보험금진전_보종 = cl_paid.develop(진전계수)['Ult'].to_frame('Value')
    기지급보험금_보종 = cl_paid.triangle_incr.fillna(0).sum(axis=1).to_frame('Value')
    지급준비금_보종 = 최종손해액_손해액진전_보종 - 기지급보험금_보종
    최종손해율_보종 = 최종손해액_손해액진전_보종/자동차_원수_계약_보종['ELP_PRM'].values.reshape(-1, 1)

    최종손해액_준비금충격1_보종 = 지급준비금_보종*위험계수 + 최종손해액_손해액진전_보종
    최종손해율_준비금충격1_보종 = 최종손해액_준비금충격1_보종/자동차_원수_계약_보종['ELP_PRM'].values.reshape(-1, 1)
    추가진전율1_보종 = 최종손해율_준비금충격1_보종-최종손해율_보종

    최종손해액_준비금충격2_보종 = (최종손해액_손해액진전_보종-최종손해액_보험금진전_보종)*위험계수 + 최종손해액_손해액진전_보종
    최종손해율_준비금충격2_보종 = 최종손해액_준비금충격2_보종/자동차_원수_계약_보종['ELP_PRM'].values.reshape(-1, 1)
    추가진전율2_보종 = 최종손해율_준비금충격2_보종-최종손해율_보종

    추가진전율_보종 = 추가진전율1_보종.merge(추가진전율2_보종, on='Accident', how='outer')
    추가진전율_보종.insert(0, 'PDGR_CD', 보종코드)

    result.append(추가진전율_보종)
df = pd.concat(result).reset_index()
df.columns = ['사고년도', '보종코드', '추가진전율1', '추가진전율2']

In [189]:
# 보종 UY 출재경과보험료 및 출재손해율
result = []
산출기간 = 5
UY_전체 = np.arange(int(기준년도)-산출기간+1, int(기준년도)+1).astype(str)

for UY in UY_전체:

    출재계약_특약_UY = 자동차_출재_계약_가공.query('UY == @UY')
    출재보상_특약_UY = 자동차_출재_보상_가공.query('UY == @UY')
    보종코드목록 = 출재계약_특약_UY.AT_PDGR_CD.drop_duplicates()

    for 보종코드 in 보종코드목록:
        출재계약_특약_UY_보종 = 출재계약_특약_UY.query('AT_PDGR_CD == @보종코드')
        출재보상_특약_UY_보종 = 출재보상_특약_UY.query('AT_PDGR_CD == @보종코드')

        # 출재 계약 (보험료)
        출재미경과_UY_국내_원수_보종 = 자동차_출재_계약_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['RN_WRTN_PRM'].sum()
        출재보험료_특약_UY_보종 = 출재계약_특약_UY_보종['RN_WRTN_PRM'].sum()

        # 출재 계약 (경과보험료)
        출재경과보험료_UY_국내_원수_보종 = 자동차_출재_계약_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['RN_ELP_PRM'].sum()
        출재경과보험료_특약_UY_보종 = 출재계약_특약_UY_보종['RN_ELP_PRM'].sum()

        # 출재 보상 (보험금)
        출재보험금_UY_국내_원수_보종 = 자동차_출재_보상_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['RN_PYN_BNF'].sum()
        출재보험금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_PYN_BNF'].sum()

        # 출재 보상 (준비금)
        출재준비금_UY_국내_원수_보종 = 자동차_출재_보상_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['RN_OST_BNF'].sum()
        출재준비금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_OST_BNF'].sum()

        # 출재 보상 (손해조사비)
        출재손조비_UY_국내_원수_보종 = 자동차_출재_보상_가공.query('UY == @UY').query('AT_PDGR_CD == @보종코드')['RN_LSAT'].sum()
        출재손조비_특약_UY_보종 = 출재보상_특약_UY_보종['RN_LSAT'].sum()

        # 출재 보상 (손해액)
        출재손해액_특약_UY_보종 = 출재보험금_특약_UY_보종 + 출재준비금_특약_UY_보종 + 출재손조비_특약_UY_보종

        # 출재 보상 (손해율)
        출재손해율_특약_UY_보종 = 출재손해액_특약_UY_보종/출재경과보험료_특약_UY_보종
        출재손해율_특약_UY_보종

        컬럼 = ['UY', '보종코드', '출재보험료', '출재경과보험료', '출재미경과보험료', '출재손해액', '출재손해율']
        데이터 = [UY, 보종코드, 출재보험료_특약_UY_보종, 출재경과보험료_특약_UY_보종, 출재보험료_특약_UY_보종-출재경과보험료_특약_UY_보종, 출재손해액_특약_UY_보종, 출재손해율_특약_UY_보종]
        dg = pd.DataFrame(데이터, 컬럼).T
        result.append(dg)

dg = pd.concat(result)

In [203]:
# 수수료 식
def auto_commission(UY, R):
    if UY == '2018':
        if R <= 0.75:
            C = 0.24
        elif R <= 0.85:
            C = 0.9945 - R
        elif R <= 0.95:
            C = 0.9950 - R
        elif R <= 0.9955:
            C = 0.9955 - R
        else:
            C = 0
    elif UY == '2017':
        if R <= 0.75:
            C = 0.24
        elif R <= 0.85:
            C = 0.9945 - R
        elif R <= 0.95:
            C = 0.9950 - R
        elif R <= 1.05:
            C = 0.9955 - R
        elif R <= 1.15:
            C = 0.9960 - R
        elif R <= 1.25:
            C = 1 - R
        else:
            C = (1-R)*0.6 + 0.1
    elif UY == '2016':
        if R <= 0.75:
            C = 0.24
        elif R <= 0.90:
            C = 0.992 - R
        elif R <= 1.10:
            C = 0.995 - R
        elif R <= 1.25:
            C = 0.998 - R
        elif R <= 1.45:
            C = (1-R)*0.9
        else:
            C = (1-R)*0.6
    elif UY == '2015':
        if R <= 0.80:
            C = 0.18
        elif R <= 0.90:
            C = 0.99 - R
        elif R <= 1.05:
            C = 0.995 - R
        elif R <= 1.30:
            C = 1.00 - R
        else:
            C = 0
    elif UY == '2014':
        if R <= 0.80:
            C = 0.18
        elif R <= 0.90:
            C = 0.99 - R
        elif R <= 1.05:
            C = 0.995 - R
        elif R <= 1.30:
            C = 1.00 - R
        else:
            C = 0
    else:
        raise Exception('유효하지 않은 UY입니다.')
    return C

In [250]:
# 수수료 증감
df2 = dg.merge(df, left_on=['UY', '보종코드'], right_on=['사고년도', '보종코드'], how='left')
df2 = df2.merge(보험가격_위험계수, on='보종코드', how='left')
df2 = df2.merge(기준손해율[['보종코드', '기준손해율']],on='보종코드', how='left')
df2 = df2.dropna(axis=0) # 근재보험특약/종합보종은 5년까지 밖에 없음

# 보험가격 충격
df2['기준손해율_보험가격충격'] = df2['기준손해율']*(1+df2['보험가격위험계수'])
df2['수수료_보험가격실적기준'] = df2['출재미경과보험료']*df2.apply(lambda df: auto_commission(df['UY'],df['기준손해율']), axis=1)
df2['수수료_보험가격충격기준'] = df2['출재미경과보험료']*df2.apply(lambda df: auto_commission(df['UY'],df['기준손해율_보험가격충격']), axis=1)
df2['수수료증감_보험가격충격'] = df2['수수료_보험가격충격기준'] - df2['수수료_보험가격실적기준']

# 준비금 충격
df2['출재손해율_준비금충격1'] = df2['출재손해율'] + df2['추가진전율1']
df2['출재손해율_준비금충격2'] = df2['출재손해율'] + df2['추가진전율2']
df2['수수료_준비금실적기준'] = df2['출재경과보험료']*df2.apply(lambda df: auto_commission(df['UY'],df['출재손해율']), axis=1)
df2['수수료_준비금충격1기준'] = df2['출재경과보험료']*df2.apply(lambda df: auto_commission(df['UY'],df['출재손해율_준비금충격1']), axis=1)
df2['수수료_준비금충격2기준'] = df2['출재경과보험료']*df2.apply(lambda df: auto_commission(df['UY'],df['출재손해율_준비금충격2']), axis=1)
df2['수수료증감_준비금충격1'] = df2['수수료_준비금충격1기준'] - df2['수수료_준비금실적기준']
df2['수수료증감_준비금충격2'] = df2['수수료_준비금충격2기준'] - df2['수수료_준비금실적기준']
df2

,UY,보종코드,출재보험료,출재경과보험료,출재미경과보험료,출재손해액,출재손해율,사고년도,추가진전율1,추가진전율2,...,수수료_보험가격실적기준,수수료_보험가격충격기준,수수료증감_보험가격충격,출재손해율_준비금충격1,출재손해율_준비금충격2,수수료_준비금실적기준,수수료_준비금충격1기준,수수료_준비금충격2기준,수수료증감_준비금충격1,수수료증감_준비금충격2
0,2014,B005,219280710,219280710,0,401270,0.00,2014,0.01,0.01,...,0.00,-0.00,-0.00,0.01,0.01,"39,470,527.80","39,470,527.80","39,470,527.80",0.00,0.00
1,2014,B006,17745234,17745234,0,2846484,0.16,2014,0.00,0.00,...,0.00,0.00,0.00,0.16,0.16,"3,194,142.12","3,194,142.12","3,194,142.12",0.00,0.00
2,2014,B007,3596950494,3596950494,0,68179262,0.02,2014,0.00,0.00,...,0.00,0.00,0.00,0.02,0.02,"647,451,088.92","647,451,088.92","647,451,088.92",0.00,0.00
3,2015,B005,4919726586,4919726586,0,816679892,0.17,2015,0.01,0.01,...,0.00,-0.00,-0.00,0.18,0.18,"885,550,785.48","885,550,785.48","885,550,785.48",0.00,0.00
4,2015,B001,-3081297,-3081297,0,27927380,-9.06,2015,0.00,0.00,...,0.00,0.00,0.00,-9.06,-9.06,"-554,633.46","-554,633.46","-554,633.46",0.00,0.00
5,2015,B007,6665510045,6665510045,0,607038316,0.09,2015,0.00,0.00,...,0.00,0.00,0.00,0.09,0.09,"1,199,791,808.10","1,199,791,808.10","1,199,791,808.10",0.00,0.00
6,2015,B002,-4002854,-4002854,0,32757142,-8.18,2015,0.00,0.00,...,0.00,0.00,0.00,-8.18,-8.18,"-720,513.72","-720,513.72","-720,513.72",0.00,0.00
7,2015,B006,333014297,333014297,0,5165048,0.02,2015,0.00,0.00,...,0.00,0.00,0.00,0.02,0.02,"59,942,573.46","59,942,573.46","59,942,573.46",0.00,0.00
8,2016,B001,4123093886,4123093886,0,7891855586,1.91,2016,0.01,0.00,...,0.00,0.00,0.00,1.92,1.92,"-2,261,257,020.00","-2,277,109,299.41","-2,273,497,616.16","-15,852,279.41","-12,240,596.16"
9,2016,B002,8273943616,8273943616,0,9655261952,1.17,2016,0.00,0.00,...,0.00,0.00,0.00,1.17,1.17,"-1,397,866,223.23","-1,400,671,668.17","-1,400,963,779.10","-2,805,444.93","-3,097,555.87"


In [254]:
df3 = df2.groupby(['보종코드'])[['수수료증감_보험가격충격', '수수료증감_준비금충격1', '수수료증감_준비금충격2']].sum().reset_index()
df3

,보종코드,수수료증감_보험가격충격,수수료증감_준비금충격1,수수료증감_준비금충격2
0,B001,"-1,051,273.26","-49,500,409.66","-37,380,472.34"
1,B002,"-1,476,422.87","-5,035,964.38","-4,756,192.44"
2,B003,"-656,043,607.34","-876,880.03","-560,764.92"
3,B004,"-667,881,373.99","-35,227.11","-23,517.06"
4,B005,-0.00,0.00,0.00
5,B006,0.00,0.00,0.00
6,B007,0.00,"-5,127,162.64","-4,674,796.90"


In [255]:
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')
    
now = datetime.now().strftime('%Y%m%d%H%M%S')
ver = 'v1.0.0'
with pd.ExcelWriter('./result/자동차_충격시나리오_{}_{}.xlsx'.format(ver, now)) as writer:
    df2.to_excel(writer, '자동차_충격시나리오_세부내역', index=False)
    df3.to_excel(writer, '자동차_충격시나리오', index=False)